In [ ]:
%reload_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

from dmsensei import DataModule, create_model
from dmsensei.config import device
from dmsensei.core.callbacks import WandbFitLogger, KaggleLogger
from lightning.pytorch.callbacks import LearningRateMonitor
from lightning.pytorch import Trainer
from dmsensei.config import device
import sys
import os
from lightning.pytorch.loggers import WandbLogger
import torch, wandb
import numpy as np

device = 'cuda'

# Create dataset
dm = DataModule(
    name=["ribo-kaggle"],
    data_type=["dms", "shape",'structure'],
    force_download=False,
    batch_size=32,
    num_workers=0,
    train_split=256,
    valid_split=0,
    predict_split=0,
    overfit_mode=True,
    shuffle_valid=False,
)

dm.setup('fit')

In [ ]:

import torch
import torch.nn.functional as F

lr = 5E-4
model = create_model(
    model="transformer",
    # weight_data=True,
    ntoken=5,
    d_model=64,
    nhead=16,
    d_hid=128,
    nlayers=2,
    dropout=0.0,
    lr=lr,
    weight_decay=0,
    gamma=0.997,
    c_z=8,
).to(device)

In [ ]:
for batch in dm.train_dataloader():
    model.training_step(batch, 0)
    break

In [ ]:
wandb.init(project="dmsensei", name='debug-structure')
trainer = Trainer(
    max_epochs=10000,
    callbacks=[
        WandbFitLogger(dm=dm)
    ],
    logger=WandbLogger(),
    )

trainer.fit(model, dm)

In [ ]:
pred, true = batch.get_pairs('structure')